<img src="https://gitlab.inf.unibz.it/earth_observation_public/eurac-openeo-examples/-/raw/main/python/aux_data/eurac_EO.png"
     alt="EURAC EO logo"
     style="float: left; margin-right: 10px; max-width: 300px" />

# openEO - Cloud Free RGB Composite based on Sentinel-2 L2A
Author michele.claus@eurac.edu

Updated: 2023/05/15

In this notebook we will use openEO to generate a cloud free RGB composite based on Sentinel-2 L2A data, using the S2_L2A_ALPS collection, which provides Sentinel-2 L2A data with a cloud mask layer generated with s2cloudless over the Alps.

In [ ]:
import openeo
from openeo.rest.datacube import PGNode, THIS
from openeo.processes import mean, eq, median, or_, array_element, clip
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt

Connect to the EURAC openEO back-end

In [ ]:
euracEndpoint = "https://openeo.eurac.edu"
conn = openeo.connect(euracEndpoint).authenticate_oidc(client_id="openEO_PKCE")

You can get detailed metadata about the Sentinel-2 collection with the following code:

In [ ]:
conn.describe_collection("S2_L2A_ALPS")

Set the temporal extent we want to consider

In [ ]:
temporal_extent = ["2021-05-01","2021-09-01"]

Set the spatial extent (area around Bolzano as an example)

In [ ]:
spatial_extent = { "west": 11.071590350000491,"east": 11.588542226063511,"south": 46.40991407331259,"north": 46.56346581927167}

Set the collection name for the Sentinel-2 data

In [ ]:
S2_collection = "S2_L2A_ALPS"

Set the bands we want to load

In [ ]:
bands = ["B02","B03","B04","CLOUD_MASK"]

Load the S2 data:

In [ ]:
S2_data = conn.load_collection(S2_collection,
                               temporal_extent=temporal_extent,
                               bands=bands,
                               spatial_extent=spatial_extent)#.filter_bbox(spatial_extent)

We need to mask out (setting it to not a number) the zero values, i.e. where there is no data. 

In [ ]:
data_mask = S2_data.filter_bands("B04").reduce_dimension(dimension="bands",reducer = lambda value: eq(array_element(value,0),0))
S2_L2A_masked = S2_data.mask(data_mask)

We create a mask for clouds (CLOUD_MASK == 1).

In [ ]:
cloud_mask = S2_data.filter_bands("CLOUD_MASK").reduce_dimension(dimension="bands",reducer = lambda value: eq(array_element(value,0),1))

Apply the mask to the S2 data

In [ ]:
S2_L2A_masked = S2_L2A_masked.filter_bands(["B02","B03","B04"]).mask(cloud_mask)

Compute the median over time

In [ ]:
S2_data_masked_median = S2_L2A_masked.reduce_dimension(dimension="DATE",reducer=median)

Clip the data between 0 and 1800 for a better visualization

In [ ]:
S2_data_masked_median_clipped = S2_data_masked_median.apply(lambda value: value.clip(0,1800))

Save the result as geoTIFF

In [ ]:
S2_data_masked_median_tiff = S2_data_masked_median_clipped.save_result(format="GTIFF")

Start a batch job and wait until it is marked as finished

In [ ]:
job_title = "S2_L2A_RGB_cloud_free_composite2"
job = conn.create_job(S2_data_masked_median_tiff,title=job_title)
job_id = job.job_id
print("Batch job created with id: ",job_id)
job.start_job()

In [ ]:
job = conn.job(job_id)
job

Download the result

In [ ]:
results = job.get_results()
results.download_files('./RGB_results/')

Tone mapping function for a nicer RGB visualization:

In [ ]:
def tone_mapping(B04,B03,B02):
    red = B04.values
    green = B03.values
    blue = B02.values
    red = (red+1)/1733*255
    green = (green+1)/1630*255
    blue = (blue+1)/1347*255
    red = np.clip(red,0,255).astype(np.uint8)
    green = np.clip(green,0,255).astype(np.uint8)
    blue = np.clip(blue,0,255).astype(np.uint8)
    brg = np.zeros((red.shape[0],red.shape[1],3),dtype=np.uint8)
    brg[:,:,0] = red
    brg[:,:,1] = green
    brg[:,:,2] = blue
    return brg

Open the result

In [ ]:
result = rioxarray.open_rasterio('RGB_results/result.tiff')
result

In [ ]:
rgb = tone_mapping(result[2],result[1],result[0])
fig, ax = plt.subplots(figsize=(18, 7))
ax.imshow(rgb)
plt.show()

But what if we would like to do also the tone mapping part directly in openEO?

The syntax wouldn't be so different!

In [ ]:
from openeo.processes import clip

red = S2_data_masked_median.band("B04")
green = S2_data_masked_median.band("B03")
blue = S2_data_masked_median.band("B02")

red = (red+1)/1733*255
green = (green+1)/1630*255
blue = (blue+1)/1347*255

red = red.add_dimension(name="bands",label="red",type="bands")
green = green.add_dimension(name="bands",label="green",type="bands")
blue = blue.add_dimension(name="bands",label="blue",type="bands")
rgb_openeo = red.merge_cubes(green).merge_cubes(blue)

rgb_openeo = rgb_openeo.apply(lambda x: clip(x,0,255))

Download the resulting PNG image directly using a synchronous call:

In [ ]:
rgb_openeo.download('RGB_results/openeo_tone_mapping.png')

Move around color channels to get RGB, the order that matplotlib expects:

In [ ]:
rgb2 = plt.imread('RGB_results/openeo_tone_mapping.png')
rgb2 = rgb2[:,:,::-1]

Visualize both results:

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(18, 7))
ax[0].imshow(rgb)
ax[0].set_title('RGB numpy tone mapping')
ax[1].imshow(rgb2)
ax[1].set_title('RGB openEO tone mapping')
plt.show()